In [1]:
## Bring in mod, seaborn
import numpy as np, pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(font_scale=1)
## Set tick style
sns.set(style="ticks", context="talk")
plt.style.use(['tableau-colorblind10'])

In [23]:
## Bring in median filtered data
## set path to file, should end in, "curation_baselined_median21.csv'
file_path = '../FILES/Tecan_qtlruns_Jan82019_curated_baselined_median51.csv'
df = pd.read_csv(file_path,index_col=0)
## Set strain to index
df.index = df.strain
dfcol = df.columns.tolist()
time_ix = dfcol.index('0s')
times_ixs = dfcol[time_ix:]
x = np.array([int(a[:-1]) for a in times_ixs])/(60**2)
dx = np.round(np.max(np.diff(x)),2)
## Set experimental conditions
temps = [30,37,39]
drugs = df.amphB.unique()

In [3]:
## Set path to genotype data
geno_path = '../FILES/CDx-ill-SNP-INDEL-df-104-blocked.csv'
geno_df = pd.read_csv(geno_path)
strain_ix = geno_df.columns.tolist().index('Block') + 1
geno_strains = np.array(geno_df.columns.tolist()[strain_ix:])

In [4]:
## Bring in info data frame
info_path = '../FILES/OLD/OCT2018/CDx_PB_Ill_Sample_Info.csv'
infodf = pd.read_csv(info_path)

In [38]:
## Define parental units
parents = ['XL280a','XL280alpha','SS-A837']
uni_seg = infodf[(infodf.Strain.isin(geno_strains)) & (infodf.Code==2)].Strain.tolist()
bis_seg = infodf[(infodf.Strain.isin(geno_strains)) & (infodf.Code==1)].Strain.tolist()
print(len(parents),len(uni_seg),len(bis_seg))

3 55 46


In [6]:
## Check work
assert len(geno_strains) == len(parents) + len(uni_seg) + len(bis_seg)

In [89]:
for t in temps:
    for d in drugs:
        dft = df[(df.temp==t) & (df.amphB==d)]
        cumsumdft = dft[times_ixs].apply(np.cumsum,axis=1)*dx
        auc = cumsumdft[times_ixs[-1]]
        auc_m = auc.groupby(auc.index).agg('mean')
        R2 = np.diff(auc_m.T[[parents[0],parents[-1]]].values)**2
        R1 = np.diff(auc_m.T[[parents[1],parents[-1]]].values)**2
        bi_en = R2/(4*auc_m.T[bis_seg].var(ddof=1))
        un_en = R1/(4*auc_m.T[uni_seg].var(ddof=1))
        print(t,d,":",bi_en,un_en,'\n')

30 0.0 : [2.25766845] [0.18612994] 

30 0.075 : [0.25066056] [0.12251262] 

30 0.125 : [1.19914649] [0.99843546] 

30 0.175 : [0.97322006] [0.32444228] 

37 0.0 : [3.91059224] [3.46669225] 

37 0.075 : [1.06125443] [0.16808127] 

37 0.125 : [0.05379052] [0.14176445] 

37 0.175 : [0.38515565] [0.02475288] 

39 0.0 : [0.12378547] [0.00148899] 

39 0.075 : [0.43453612] [0.44876865] 

39 0.125 : [1.13346887] [0.21250668] 



KeyError: "None of [['XL280a', 'SS-A837']] are in the [index]"

array([2.25766845])

In [57]:
auc.T[bis_seg].var(ddof=1)*4

208.37379122016975

150.29461221831156